In [ ]:
# !pip install -q torchmetrics

In [ ]:
# !git clone https://github.com/mamintoosi-papers-codes/TGCN-PyTorch.git
# %cd TGCN-PyTorch

In [ ]:
# %run main.py --config configs/tgcn-los-pre_len1.yaml

In [ ]:
%%time

datasets = ['los', 'sz'] # los=losloop, sz=shenzhen
pred_list = [1, 2]#, 3, 4]

for  dataset in datasets:
    for pre_len in pred_list:
        %run main.py --config configs/tgcn-{dataset}-pre_len{pre_len}.yaml
        %run main.py --config configs/tgcn-{dataset}-gsl-pre_len{pre_len}.yaml
        %run main.py --config configs/tgcn-{dataset}-gsl-adj-pre_len{pre_len}.yaml

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Define the prediction lengths and measures
pred_list = [1, 2, 3, 4]
measures = ["RMSE", "MAE", "Accuracy", "R2"]
datasets = ['losloop', 'shenzhen']

# Define a color palette for each method
colors = plt.cm.tab10.colors  # Using a predefined color palette (10 distinct colors)
method_colors = {
    "gsl0": colors[0],  # T-GCN (Default)
    "gsl1": colors[1],  # T-GCN (GSL Only)
    "gsl2": colors[2],  # T-GCN (GSL + Adj)
}

# Define line styles for each method
line_styles = {
    "gsl0": "--",  # Dashed for T-GCN (Default)
    "gsl1": "-",   # Solid for T-GCN (GSL Only)
    "gsl2": ":",   # Dotted for T-GCN (GSL + Adj)
}

# Define markers for each method
markers = {
    "gsl0": "o",  # Circle for T-GCN (Default)
    "gsl1": "s",  # Square for T-GCN (GSL Only)
    "gsl2": "D",  # Diamond for T-GCN (GSL + Adj)
}

# Loop through each dataset
for dataset in datasets:
    # Loop through each measure
    for measure in measures:
        # Loop through each pre_len in pred_list
        for pre_len in pred_list:
            # Create a new figure for each pre_len and measure
            plt.figure(figsize=(12, 8))

            # Load the metrics for gsl0 (T-GCN Default) for comparison
            metrics_file_gsl0 = f"results/metrics_{dataset}_TGCN_seq12_pre{pre_len}_gsl0.csv"
            metrics_df_gsl0 = pd.read_csv(metrics_file_gsl0)

            # Loop through each method (gsl0, gsl1, gsl2)
            for method, color in method_colors.items():
                # Generate the file path dynamically based on pre_len and method
                metrics_file = f"results/metrics_{dataset}_TGCN_seq12_pre{pre_len}_{method}.csv"

                # Load the metrics CSV file
                metrics_df = pd.read_csv(metrics_file)

                # Plot the current measure for the current method
                plt.plot(
                    metrics_df["Epoch"],
                    metrics_df[measure],
                    linestyle=line_styles[method],
                    marker=markers[method],
                    color=color,
                    label=f"{method} (pre_len={pre_len})",
                )

                # Highlight the first epoch where the current method is better than T-GCN (Default)
                if method != "gsl0":  # Skip comparison for T-GCN (Default)
                    better_epoch = None
                    for epoch in range(len(metrics_df)):
                        tgcn_value = metrics_df_gsl0.loc[epoch, measure]
                        current_value = metrics_df.loc[epoch, measure]

                        # Check if the current method is better than T-GCN (Default)
                        if measure in ["RMSE", "MAE"]:
                            if current_value < tgcn_value:  # Lower is better
                                better_epoch = epoch + 1  # Epochs are 1-indexed
                                break
                        else:
                            if current_value > tgcn_value:  # Higher is better
                                better_epoch = epoch + 1
                                break

                    # Highlight the first epoch where the current method is better
                    if better_epoch is not None:
                        plt.axvline(
                            x=better_epoch,
                            color=color,
                            linestyle=":",
                            alpha=0.5,
                            label=f"{method} better at epoch {better_epoch} (pre_len={pre_len})",
                        )

            # Add labels, title, legend, and grid
            plt.xlabel("Epoch")
            plt.ylabel(measure)
            plt.title(f"{measure} over Epochs for {dataset} (pre_len={pre_len})")
            plt.legend()
            plt.grid(True)

            # Show the plot for the current pre_len and measure
            plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Define the prediction lengths and measures
pred_list = [1, 2, 3, 4]
measures = ["RMSE", "MAE", "Accuracy", "R2"]
# last_epoch_to_show = 50
# Define a color palette for each pre_len
colors = plt.cm.tab10.colors  # Using a predefined color palette (10 distinct colors)

# Loop through each measure
for measure in measures:
    # Create a new figure for each measure
    plt.figure(figsize=(12, 8))

    # Loop through each pre_len in pred_list
    for i, pre_len in enumerate(pred_list):
        # Generate the file paths dynamically based on pre_len
        metrics_file_gsl_false = f"results/metrics_losloop_TGCN_seq12_pre{pre_len}_gslFalse.csv"
        metrics_file_gsl_true = f"results/metrics_losloop_TGCN_seq12_pre{pre_len}_gslTrue.csv"

        # Load the metrics CSV files
        metrics_df_gsl_false = pd.read_csv(metrics_file_gsl_false)
        metrics_df_gsl_true = pd.read_csv(metrics_file_gsl_true)

        # Remove the rows after last_epoch_to_show
        metrics_df_gsl_true = metrics_df_gsl_true.iloc[:last_epoch_to_show]
        metrics_df_gsl_false = metrics_df_gsl_false.iloc[:last_epoch_to_show]

        # Use the same base color for both methods, but with different intensity
        base_color = colors[i % len(colors)]  # Cycle through the color palette
        color_false = base_color  # Full intensity for GSL-false
        color_true = (*base_color[:3], 0.6)  # Reduced intensity (alpha = 0.6) for GSL-true

        # Plot the current measure for the current pre_len
        # Use dashed line for GSL-false (TGCN)
        plt.plot(metrics_df_gsl_false["Epoch"], metrics_df_gsl_false[measure], linestyle='--', marker='o', color=color_false, label=f"TGCN (pre_len={pre_len})")
        # Use solid line for GSL-true (GSL-TGCN)
        plt.plot(metrics_df_gsl_true["Epoch"], metrics_df_gsl_true[measure], linestyle='-', marker='s', color=color_true, label=f"GSL-TGCN (pre_len={pre_len})")

        # Find the first epoch where GSL-true is better than GSL-false
        better_epoch = None
        for epoch in range(len(metrics_df_gsl_false)):
            tgcn_value = metrics_df_gsl_false.loc[epoch, measure]
            gsl_tgcn_value = metrics_df_gsl_true.loc[epoch, measure]

            # Check if GSL-true is better than TGCN
            if measure in ["RMSE", "MAE"]:
                if gsl_tgcn_value < tgcn_value:  # Lower is better
                    better_epoch = epoch + 1  # Epochs are 1-indexed
                    break
            else:
                if gsl_tgcn_value > tgcn_value:  # Higher is better
                    better_epoch = epoch + 1
                    break

        # Highlight the first epoch where GSL-true is better
        if better_epoch is not None : #and better_epoch <= 30:
            plt.axvline(x=better_epoch, color=color_true, linestyle=':', alpha=0.5, label=f"GSL-TGCN better at epoch {better_epoch} (pre_len={pre_len})")

    # Add labels, title, legend, and grid
    plt.xlabel("Epoch")
    plt.ylabel(measure)
    plt.title(f"{measure} over Epochs for TGCN and GSL-TGCN (losloop)")
    plt.legend()
    # plt.grid(True)

    # Limit the x-axis to the first 30 epochs
    # plt.xlim(0, 30)

    # Show the plot for the current measure
    plt.show()

In [ ]:
import pandas as pd

# Define the prediction lengths and measures
pred_list = [1, 2, 3, 4]
measures = ["RMSE", "MAE", "Accuracy", "R2"]

# Initialize a dictionary to store the results of the last epoch
results = {}

# Loop through each pre_len and measure to collect the last epoch results
for pre_len in pred_list:
    # Generate the file paths dynamically based on pre_len
    metrics_file_gsl_false = f"results/metrics_losloop_TGCN_seq12_pre{pre_len}_gslFalse.csv"
    metrics_file_gsl_true = f"results/metrics_losloop_TGCN_seq12_pre{pre_len}_gslTrue.csv"

    # Load the metrics CSV files
    metrics_df_gsl_false = pd.read_csv(metrics_file_gsl_false)
    metrics_df_gsl_true = pd.read_csv(metrics_file_gsl_true)

    # Remove the rows after last_epoch_to_show
    metrics_df_gsl_true = metrics_df_gsl_true.iloc[:last_epoch_to_show]
    metrics_df_gsl_false = metrics_df_gsl_false.iloc[:last_epoch_to_show]
    
    # Get the last epoch results for both methods
    last_epoch_false = metrics_df_gsl_false.iloc[-1]  # Last row for GSL-false
    last_epoch_true = metrics_df_gsl_true.iloc[-1]    # Last row for GSL-true

    # Store the results in the dictionary
    results[f"TGCN (pre_len={pre_len})"] = last_epoch_false[measures]
    results[f"GSL-TGCN (pre_len={pre_len})"] = last_epoch_true[measures]

# Convert the results dictionary to a DataFrame
results_df = pd.DataFrame(results).T

# Function to highlight the winner for each pre_len and measure
def highlight_winner(df):
    highlighted_df = df.copy()
    for pre_len in pred_list:
        for measure in measures:
            # Get the values for the two methods for the current pre_len
            tgcn_value = df.loc[f"TGCN (pre_len={pre_len})", measure]
            gsl_tgcn_value = df.loc[f"GSL-TGCN (pre_len={pre_len})", measure]

            # Determine the winner based on the measure
            if measure in ["RMSE", "MAE"]:
                winner_value = min(tgcn_value, gsl_tgcn_value)  # Lower is better
            else:
                winner_value = max(tgcn_value, gsl_tgcn_value)  # Higher is better

            # Highlight the winner
            if tgcn_value == winner_value:
                highlighted_df.loc[f"TGCN (pre_len={pre_len})", measure] = f"\\textbf{{{tgcn_value:.4f}}}"
            else:
                highlighted_df.loc[f"TGCN (pre_len={pre_len})", measure] = f"{tgcn_value:.4f}"

            if gsl_tgcn_value == winner_value:
                highlighted_df.loc[f"GSL-TGCN (pre_len={pre_len})", measure] = f"\\textbf{{{gsl_tgcn_value:.4f}}}"
            else:
                highlighted_df.loc[f"GSL-TGCN (pre_len={pre_len})", measure] = f"{gsl_tgcn_value:.4f}"
    return highlighted_df

# Apply the highlight function to the results DataFrame
highlighted_results = highlight_winner(results_df)

# Generate the LaTeX table
latex_table = highlighted_results.to_latex(escape=False, column_format="l" + "c" * len(measures))

# Print the LaTeX table
print(latex_table)

In [ ]:
# metrics_df_gsl_false['RMSE'], metrics_df_gsl_true['RMSE']

In [ ]:
import pandas as pd

# Define the prediction lengths and measures
pred_list = [1, 2, 3, 4]
measures = ["RMSE", "MAE", "Accuracy", "R2"]

# Initialize a dictionary to store the first epoch where GSL-true is better
first_epoch_better = {f"pre_len={pre_len}": {measure: None for measure in measures} for pre_len in pred_list}

# Loop through each pre_len and measure to find the first epoch where GSL-true is better
for pre_len in pred_list:
    # Generate the file paths dynamically based on pre_len
    metrics_file_gsl_false = f"results/metrics_losloop_TGCN_seq12_pre{pre_len}_gslFalse.csv"
    metrics_file_gsl_true = f"results/metrics_losloop_TGCN_seq12_pre{pre_len}_gslTrue.csv"

    # Load the metrics CSV files
    metrics_df_gsl_false = pd.read_csv(metrics_file_gsl_false)
    metrics_df_gsl_true = pd.read_csv(metrics_file_gsl_true)

    # Remove the rows after last_epoch_to_show
    metrics_df_gsl_true = metrics_df_gsl_true.iloc[:last_epoch_to_show]
    metrics_df_gsl_false = metrics_df_gsl_false.iloc[:last_epoch_to_show]

    # Loop through each measure
    for measure in measures:
        # Initialize a flag to track if GSL-true has become better
        better_epoch = None

        # Loop through each epoch
        for epoch in range(len(metrics_df_gsl_false)):
            tgcn_value = metrics_df_gsl_false.loc[epoch, measure]
            gsl_tgcn_value = metrics_df_gsl_true.loc[epoch, measure]

            # Check if GSL-true is better than TGCN
            if measure in ["RMSE", "MAE"]:
                if gsl_tgcn_value < tgcn_value:  # Lower is better
                    better_epoch = epoch + 1  # Epochs are 1-indexed
                    break
            else:
                if gsl_tgcn_value > tgcn_value:  # Higher is better
                    better_epoch = epoch + 1
                    break

        # Store the result
        first_epoch_better[f"pre_len={pre_len}"][measure] = better_epoch if better_epoch is not None else "-"

# Convert the results dictionary to a DataFrame
results_df = pd.DataFrame(first_epoch_better).T

# Generate the LaTeX table
latex_table = results_df.to_latex(escape=False, column_format="l" + "c" * len(measures))

# Print the LaTeX table
print(latex_table)

In [ ]:
%%time
pred_list = [1, 2, 3, 4]
for pre_len in pred_list:
    %run main.py --config configs/tgcn-sz-pre_len{pre_len}.yaml --log_path train-tgcn-sz-pre_len{pre_len}.log
    %run main.py --config configs/tgcn-sz-gsl-pre_len{pre_len}.yaml --log_path train-tgcn-sz-gsl-pre_len{pre_len}.log

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Define the prediction lengths and measures
pred_list = [1, 2, 3, 4]
measures = ["RMSE", "MAE", "Accuracy", "R2"]

# Define a color palette for each pre_len
colors = plt.cm.tab10.colors  # Using a predefined color palette (10 distinct colors)

# Loop through each measure
for measure in measures:
    # Create a new figure for each measure
    plt.figure(figsize=(12, 8))

    # Loop through each pre_len in pred_list
    for i, pre_len in enumerate(pred_list):
        # Generate the file paths dynamically based on pre_len
        metrics_file_gsl_false = f"results/metrics_shenzhen_TGCN_seq12_pre{pre_len}_gslFalse.csv"
        metrics_file_gsl_true = f"results/metrics_shenzhen_TGCN_seq12_pre{pre_len}_gslTrue.csv"

        # Load the metrics CSV files
        metrics_df_gsl_false = pd.read_csv(metrics_file_gsl_false)
        metrics_df_gsl_true = pd.read_csv(metrics_file_gsl_true)

        # Remove the rows after last_epoch_to_show
        metrics_df_gsl_true = metrics_df_gsl_true.iloc[:last_epoch_to_show]
        metrics_df_gsl_false = metrics_df_gsl_false.iloc[:last_epoch_to_show]

        # Use the same base color for both methods, but with different intensity
        base_color = colors[i % len(colors)]  # Cycle through the color palette
        color_false = base_color  # Full intensity for GSL-false
        color_true = (*base_color[:3], 0.6)  # Reduced intensity (alpha = 0.6) for GSL-true

        # Plot the current measure for the current pre_len
        # Use dashed line for GSL-false (TGCN)
        plt.plot(metrics_df_gsl_false["Epoch"], metrics_df_gsl_false[measure], linestyle='--', marker='o', color=color_false, label=f"TGCN (pre_len={pre_len})")
        # Use solid line for GSL-true (GSL-TGCN)
        plt.plot(metrics_df_gsl_true["Epoch"], metrics_df_gsl_true[measure], linestyle='-', marker='s', color=color_true, label=f"GSL-TGCN (pre_len={pre_len})")

        # Find the first epoch where GSL-true is better than GSL-false
        better_epoch = None
        for epoch in range(len(metrics_df_gsl_false)):
            tgcn_value = metrics_df_gsl_false.loc[epoch, measure]
            gsl_tgcn_value = metrics_df_gsl_true.loc[epoch, measure]

            # Check if GSL-true is better than TGCN
            if measure in ["RMSE", "MAE"]:
                if gsl_tgcn_value < tgcn_value:  # Lower is better
                    better_epoch = epoch + 1  # Epochs are 1-indexed
                    break
            else:
                if gsl_tgcn_value > tgcn_value:  # Higher is better
                    better_epoch = epoch + 1
                    break

        # Highlight the first epoch where GSL-true is better
        if better_epoch is not None: # and better_epoch <= 30:
            plt.axvline(x=better_epoch, color=color_true, linestyle=':', alpha=0.5, label=f"GSL-TGCN better at epoch {better_epoch} (pre_len={pre_len})")

    # Add labels, title, legend, and grid
    plt.xlabel("Epoch")
    plt.ylabel(measure)
    plt.title(f"{measure} over Epochs for TGCN and GSL-TGCN (shenzhen)")
    plt.legend()
    # plt.grid(True)

    # Limit the x-axis to the first 30 epochs
    # plt.xlim(0, 30)

    # Show the plot for the current measure
    plt.show()

In [ ]:
import pandas as pd

# Define the prediction lengths and measures
pred_list = [1, 2, 3, 4]
measures = ["RMSE", "MAE", "Accuracy", "R2"]

# Initialize a dictionary to store the results of the last epoch
results = {}

# Loop through each pre_len and measure to collect the last epoch results
for pre_len in pred_list:
    # Generate the file paths dynamically based on pre_len
    metrics_file_gsl_false = f"results/metrics_shenzhen_TGCN_seq12_pre{pre_len}_gslFalse.csv"
    metrics_file_gsl_true = f"results/metrics_shenzhen_TGCN_seq12_pre{pre_len}_gslTrue.csv"

    # Load the metrics CSV files
    metrics_df_gsl_false = pd.read_csv(metrics_file_gsl_false)
    metrics_df_gsl_true = pd.read_csv(metrics_file_gsl_true)

    # Remove the rows after last_epoch_to_show
    metrics_df_gsl_true = metrics_df_gsl_true.iloc[:last_epoch_to_show]
    metrics_df_gsl_false = metrics_df_gsl_false.iloc[:last_epoch_to_show]
    
    # Get the last epoch results for both methods
    last_epoch_false = metrics_df_gsl_false.iloc[-1]  # Last row for GSL-false
    last_epoch_true = metrics_df_gsl_true.iloc[-1]    # Last row for GSL-true

    # Store the results in the dictionary
    results[f"TGCN (pre_len={pre_len})"] = last_epoch_false[measures]
    results[f"GSL-TGCN (pre_len={pre_len})"] = last_epoch_true[measures]

# Convert the results dictionary to a DataFrame
results_df = pd.DataFrame(results).T

# Function to highlight the winner for each pre_len and measure
def highlight_winner(df):
    highlighted_df = df.copy()
    for pre_len in pred_list:
        for measure in measures:
            # Get the values for the two methods for the current pre_len
            tgcn_value = df.loc[f"TGCN (pre_len={pre_len})", measure]
            gsl_tgcn_value = df.loc[f"GSL-TGCN (pre_len={pre_len})", measure]

            # Determine the winner based on the measure
            if measure in ["RMSE", "MAE"]:
                winner_value = min(tgcn_value, gsl_tgcn_value)  # Lower is better
            else:
                winner_value = max(tgcn_value, gsl_tgcn_value)  # Higher is better

            # Highlight the winner
            if tgcn_value == winner_value:
                highlighted_df.loc[f"TGCN (pre_len={pre_len})", measure] = f"\\textbf{{{tgcn_value:.4f}}}"
            else:
                highlighted_df.loc[f"TGCN (pre_len={pre_len})", measure] = f"{tgcn_value:.4f}"

            if gsl_tgcn_value == winner_value:
                highlighted_df.loc[f"GSL-TGCN (pre_len={pre_len})", measure] = f"\\textbf{{{gsl_tgcn_value:.4f}}}"
            else:
                highlighted_df.loc[f"GSL-TGCN (pre_len={pre_len})", measure] = f"{gsl_tgcn_value:.4f}"
    return highlighted_df

# Apply the highlight function to the results DataFrame
highlighted_results = highlight_winner(results_df)

# Generate the LaTeX table
latex_table = highlighted_results.to_latex(escape=False, column_format="l" + "c" * len(measures))

# Print the LaTeX table
print(latex_table)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Define the prediction lengths to loop through
pred_list = [1, 2, 3, 4]

# Create a figure to hold all the plots
plt.figure(figsize=(12, 8))

# Loop through each pre_len in pred_list
for pre_len in pred_list:
    # Generate the file paths dynamically based on pre_len
    metrics_file_gsl_false = f"results/metrics_shenzhen_TGCN_seq12_pre{pre_len}_gslFalse.csv"
    metrics_file_gsl_true = f"results/metrics_shenzhen_TGCN_seq12_pre{pre_len}_gslTrue.csv"

    # Load the metrics CSV files
    metrics_df_gsl_false = pd.read_csv(metrics_file_gsl_false)
    metrics_df_gsl_true = pd.read_csv(metrics_file_gsl_true)

    # Plot RMSE values for the current pre_len
    plt.plot(metrics_df_gsl_false["Epoch"], metrics_df_gsl_false["RMSE"], marker='o', label=f"TGCN (pre_len={pre_len})")
    plt.plot(metrics_df_gsl_true["Epoch"], metrics_df_gsl_true["RMSE"], marker='o', label=f"GSL-TGCN (pre_len={pre_len})")

# Add labels, title, legend, and grid
plt.xlabel("Epoch")
plt.ylabel("RMSE")
plt.title("RMSE over Epochs for TGCN and GSL-TGCN (shenzhen)")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
%%time
%run main.py --config configs/gcn-los.yaml --log_path train-gcn-los.log

Losloop

In [ ]:
%%time
%run main.py --config configs/gcn-los-gsl.yaml --log_path train-gcn-los-gsl.log

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the metrics CSV files
metrics_file_gsl_false = "results/metrics_losloop_GCN_seq12_pre1_gslFalse.csv"
metrics_file_gsl_true = "results/metrics_losloop_GCN_seq12_pre1_gslTrue.csv"

metrics_df_gsl_false = pd.read_csv(metrics_file_gsl_false)
metrics_df_gsl_true = pd.read_csv(metrics_file_gsl_true)

# Plot RMSE values
plt.figure(figsize=(10, 6))
plt.plot(metrics_df_gsl_false["Epoch"], metrics_df_gsl_false["RMSE"], marker='o', label="GCN")
plt.plot(metrics_df_gsl_true["Epoch"], metrics_df_gsl_true["RMSE"], marker='o', label="GSL-GCN")
plt.xlabel("Epoch")
plt.ylabel("RMSE")
plt.title("RMSE over Epochs for GCN and GSL-GCN")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
%%time
%run main.py --config configs/gcn-sz.yaml --log_path train-gcn-sz.log

In [ ]:
%%time
%run main.py --config configs/gcn-sz-gsl.yaml --log_path train-gcn-sz-gsl.log

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the metrics CSV files
metrics_file_gsl_false = "results/metrics_shenzhen_GCN_seq12_pre1_gslFalse.csv"
metrics_file_gsl_true = "results/metrics_shenzhen_GCN_seq12_pre1_gslTrue.csv"

metrics_df_gsl_false = pd.read_csv(metrics_file_gsl_false)
metrics_df_gsl_true = pd.read_csv(metrics_file_gsl_true)

# Plot RMSE values
plt.figure(figsize=(10, 6))
plt.plot(metrics_df_gsl_false["Epoch"], metrics_df_gsl_false["RMSE"], marker='o', label="GCN")
plt.plot(metrics_df_gsl_true["Epoch"], metrics_df_gsl_true["RMSE"], marker='o', label="GSL-GCN")
plt.xlabel("Epoch")
plt.ylabel("RMSE")
plt.title("RMSE over Epochs for GCN and GSL-GCN")
plt.legend()
plt.grid(True)
plt.show()

SZ

In [ ]:
%%time
%run main.py --config configs/tgcn-sz.yaml --log_path train-tgcn-sz.log

SZ-GSL

In [ ]:
%%time
%run main.py --config configs/tgcn-sz-gsl.yaml --log_path train-tgcn-sz-gsl.log

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the metrics CSV files
metrics_file_gsl_false = "results/metrics_shenzhen_TGCN_seq12_pre1_gslFalse.csv"
metrics_file_gsl_true = "results/metrics_shenzhen_TGCN_seq12_pre1_gslTrue.csv"

metrics_df_gsl_false = pd.read_csv(metrics_file_gsl_false)
metrics_df_gsl_true = pd.read_csv(metrics_file_gsl_true)

# Plot RMSE values
plt.figure(figsize=(10, 6))
plt.plot(metrics_df_gsl_false["Epoch"], metrics_df_gsl_false["RMSE"], marker='o', label="TGCN")
plt.plot(metrics_df_gsl_true["Epoch"], metrics_df_gsl_true["RMSE"], marker='o', label="GSL-TGCN")
plt.xlabel("Epoch")
plt.ylabel("RMSE")
plt.title("RMSE over Epochs for TGCN and GSL-TGCN (shenzhen)")
plt.legend()
plt.grid(True)
plt.show()

Los

In [ ]:
%%time
%run main.py --config configs/tgcn-los.yaml --log_path train-tgcn-los.log

Los-GSL

In [ ]:
%%time
%run main.py --config configs/tgcn-los-gsl.yaml --log_path train-tgcn-los-gsl.log

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the metrics CSV files
metrics_file_gsl_false = "results/metrics_losloop_TGCN_seq12_pre1_gslFalse.csv"
metrics_file_gsl_true = "results/metrics_losloop_TGCN_seq12_pre1_gslTrue.csv"

metrics_df_gsl_false = pd.read_csv(metrics_file_gsl_false)
metrics_df_gsl_true = pd.read_csv(metrics_file_gsl_true)

# Plot RMSE values
plt.figure(figsize=(10, 6))
plt.plot(metrics_df_gsl_false["Epoch"], metrics_df_gsl_false["RMSE"], marker='o', label="TGCN")
plt.plot(metrics_df_gsl_true["Epoch"], metrics_df_gsl_true["RMSE"], marker='o', label="GSL-TGCN")
plt.xlabel("Epoch")
plt.ylabel("RMSE")
plt.title("RMSE over Epochs for TGCN and GSL-TGCN (losloop)")
plt.legend()
plt.grid(True)
plt.show()